In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

# Import your CRUD Python module (assuming it's named animal_shelter and contains the AnimalShelter class)
from animal_shelter import AnimalShelter

# Data Manipulation / Model
username = "aacuser"
password = "test"
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# Retrieve data from MongoDB and prepare the DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'my-image.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

app.layout = html.Div([
    # Logo and dashboard title
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px', 'width': '100px'}),
        html.B(html.H1('Austin\'s SNHU CS-340 Green Dashboard', style={'color': 'green'}))
    ]),
    html.Hr(),
    # Interactive Filtering Options Placeholder
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'OPT2'},
                {'label': 'Mountain Rescue', 'value': 'OPT3'},
                {'label': 'Disaster Rescue', 'value': 'OPT4'},
                {'label': 'Reset', 'value': 'OPT1'},
                
            ],
            value='OPT1',  # Default value
            labelStyle={'display': 'block'}
        ),
        # You can add more interactive components here (e.g., dcc.Dropdown, dcc.Checklist)
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
        # Add more DataTable properties as needed to make it user-friendly
    ),
    html.Br(),
    html.Hr(),
    # Side-by-side charts and maps
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-container', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6'),
    ])
])
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Define your query based on the filter_type
    if filter_type == 'OPT2':
        # Query for Water Rescue
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr", "Newfoundland" ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'OPT3':
        # Query for Mountain Rescue
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler" ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'OPT4':
        # Query for Disaster Rescue
        query = {
            "breed": {"$in": ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler" ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    else:
        # Reset that queries all
        query = {}

    # Use the shelter.read() method with the constructed query to filter the data
    filtered_data = pd.DataFrame.from_records(shelter.read(query))

    # It's important to remove the '_id' column to avoid issues with data_table
    if '_id' in filtered_data.columns:
        filtered_data.drop(columns=['_id'], inplace=True)

    # Convert the DataFrame to dictionary format for Dash DataTable
    data = filtered_data.to_dict('records')

    return data

@app.callback(
    Output('graph-container', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData, **kwargs):
    # Guard clause for empty or None viewData
    if not viewData:
        return html.Div()  # Return an empty div if no data
    
    # Convert the viewData into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Ensure 'breed' column exists
    if 'breed' not in dff.columns:
        return html.Div()  # Return an empty div if 'breed' column is missing
    
    # Generate pie chart
    breed_counts = dff['outcome_type'].value_counts().reset_index()
    breed_counts.columns = ['outcome_type', 'count']
    fig = px.pie(breed_counts, values='count', names='outcome_type', title='Distribution of outcome_type')
    
    # Return a list containing the new Graph component
    # This replaces any existing children in the 'graph-container'
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns, **kwargs):
    # Check if selected_columns is None or empty and return a default style or empty list
    if not selected_columns:
        return []
    
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index, **kwargs):
    if viewData is None or index is None or len(index) == 0:
        return None
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    # Return the Leaflet map component with a marker set to the selected row's location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]



app.run_server()


Dash app running on http://127.0.0.1:10029/


/home/austinblanton_snhu/.local/lib/python3.9/site-packages/dash/dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

